<a href="https://colab.research.google.com/github/hangj11/AirSim/blob/master/fish_detection_ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x
!pip install tf_slim

TensorFlow is already loaded. Please restart the runtime to change versions.


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# this code is pulling the repo into your google drive once it is pulled you dont have to run this code again
%cd /content/gdrive/My Drive
! git clone https://github.com/kwea123/fish_detection

/content/gdrive/My Drive
fatal: destination path 'fish_detection' already exists and is not an empty directory.


In [ ]:
# this code is pulling the repo into your google drive once it is pulled you dont have to run this code again
%cd /content/gdrive/My Drive
!git clone --quiet https://github.com/tensorflow/models.git

/content/gdrive/My Drive
fatal: destination path 'models' already exists and is not an empty directory.


In [ ]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

%cd /content/gdrive/My Drive/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/gdrive/My Drive/models/research/:/content/gdrive/My Drive/models/research/slim/'

!python "/content/gdrive/My Drive/models/research/object_detection/builders/model_builder_test.py"

/content/gdrive/My Drive/models/research
Traceback (most recent call last):
  File "/content/gdrive/My Drive/models/research/object_detection/builders/model_builder_test.py", line 21, in <module>
    from object_detection.builders import model_builder
  File "/content/gdrive/My Drive/models/research/object_detection/builders/model_builder.py", line 37, in <module>
    from object_detection.meta_architectures import deepmac_meta_arch
  File "/content/gdrive/My Drive/models/research/object_detection/meta_architectures/deepmac_meta_arch.py", line 28, in <module>
    import tensorflow_io as tfio  # pylint:disable=g-import-not-at-top
ModuleNotFoundError: No module named 'tensorflow_io'


In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image
import time
import cv2
import matplotlib.pyplot as plt

In [ ]:
class Object_Detector():
    def __init__(self, model_path):
        self.__load_model(model_path)
        print('model loaded')

    def __load_model(self, model_path):
        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            with tf.gfile.GFile(model_path, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')

        config = tf.ConfigProto()
        config.gpu_options.allow_growth= True

        with self.detection_graph.as_default():
            self.sess = tf.Session(config=config, graph=self.detection_graph)
            self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
            self.detection_boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
            self.detection_scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
            self.detection_classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
            self.num_detections = self.detection_graph.get_tensor_by_name('num_detections:0')

        # load label_dict
        self.label_dict = {1: 'fish'}
        
        # warmup
        self.detect_image(np.ones((600, 600, 3)))

    def detect_image(self, image_np, score_thr=0.5, print_time=False):
        image_w, image_h = image_np.shape[1], image_np.shape[0]
    
        # Actual detection.
        t = time.time()
        (boxes, scores, classes, num) = self.sess.run(
          [self.detection_boxes, self.detection_scores, self.detection_classes, self.num_detections],
          feed_dict={self.image_tensor: np.expand_dims(image_np, axis=0)})
        if print_time:
            print('detection time :', time.time()-t)
        # Visualization of the results of a detection.
        for i, box in enumerate(boxes[scores>score_thr]):
            top_left = (int(box[1]*image_w), int(box[0]*image_h))
            bottom_right = (int(box[3]*image_w), int(box[2]*image_h))
            cv2.rectangle(image_np, top_left, bottom_right, (0,255,0), 3)
            cv2.putText(image_np, self.label_dict[int(classes[0,i])], top_left, cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

        return image_np

In [ ]:
MODEL_PATH = '/content/gdrive/My Drive/fish_detection/fish_inception_v2_graph/frozen_inference_graph.pb'
object_detector = Object_Detector(MODEL_PATH)

AttributeError: ignored

In [ ]:
img = cv2.imread('/content/gdrive/My Drive/fish_detection/test_images/fish2.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_ = object_detector.detect_image(img, score_thr=0.2)
plt.figure(figsize=(20, 10))
plt.imshow(img_)

NameError: ignored

In [ ]:
#

cap = cv2.VideoCapture("/content/gdrive/My Drive/fish/b9.mp4")

if cap.isOpened():
  width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # float
  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # float
  fps = int(cap.get(cv2.CAP_PROP_FPS))
  frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('/content/gdrive/My Drive/output_result.mp4',fourcc, fps, (width, height))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img_ = object_detector.detect_image(img, score_thr=0.2)
        out.write(img_)
    else:
        break

cap.release()
out.release()

NameError: ignored